# Understanding the Model and Data

## 2.1 Time Series Properties Analysis:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

# Load data
df = pd.read_csv('BrentOilPrices.csv', parse_dates=['Date'])
df = df.sort_values('Date')

# 1. Trend Analysis
plt.figure(figsize=(14, 6))
plt.plot(df['Date'], df['Price'])
plt.title('Brent Oil Price Trend (1987-2022)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.grid(True)
plt.show()

# Rolling statistics
df['30_day_avg'] = df['Price'].rolling(window=30).mean()
df['30_day_std'] = df['Price'].rolling(window=30).std()

# 2. Stationarity Testing
def test_stationarity(timeseries):
    # Perform Dickey-Fuller test
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries.dropna(), autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], 
                        index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key, value in dftest[4].items():
        dfoutput[f'Critical Value ({key})'] = value
    print(dfoutput)
    
    # Determine stationarity
    if dftest[1] <= 0.05:
        print("Data is stationary")
    else:
        print("Data is non-stationary")

print("Testing stationarity of raw prices:")
test_stationarity(df['Price'])

# 3. Log returns for stationarity
df['Log_Return'] = np.log(df['Price']) - np.log(df['Price'].shift(1))

print("\nTesting stationarity of log returns:")
test_stationarity(df['Log_Return'].dropna())

# 4. Volatility Patterns
plt.figure(figsize=(14, 6))
plt.subplot(2, 1, 1)
plt.plot(df['Date'], df['Log_Return'])
plt.title('Log Returns - Volatility Clustering')
plt.xlabel('Date')
plt.ylabel('Log Return')

plt.subplot(2, 1, 2)
df['Log_Return'].rolling(window=30).std().plot()
plt.title('30-Day Rolling Volatility')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.tight_layout()
plt.show()

## Build Bayesian Change Point Model (PyMC)

In [ ]:
import pymc as pm
import arviz as az

price_data = df['Price'].values
n = len(price_data)

with pm.Model() as model:
    # Prior for change point (tau)
    tau = pm.DiscreteUniform("tau", lower=0, upper=n-1)
    
    # Means before and after change point
    mu1 = pm.Normal("mu1", mu=price_data.mean(), sigma=10)
    mu2 = pm.Normal("mu2", mu=price_data.mean(), sigma=10)
    
    # Switch function
    mean = pm.math.switch(tau > np.arange(n), mu1, mu2)
    
    # Likelihood
    likelihood = pm.Normal("likelihood", mu=mean, sigma=1, observed=price_data)
    
    # Sampling
    trace = pm.sample(2000, tune=1000, return_inferencedata=True)

## Interpret Model Output

In [ ]:
# Check convergence
az.summary(trace)

# Plot trace
az.plot_trace(trace)

# Plot posterior of tau
az.plot_posterior(trace, var_names=["tau"])

Modeling Implications:

Non-stationarity in prices → Need differencing or log returns

Volatility clustering → Consider GARCH models for advanced analysis

Multiple regimes → Markov-switching models could be beneficial

2.2 Change Point Models Explanation:
Purpose in Context:
Change point models identify structural breaks where statistical properties of a time series change abruptly. For Brent oil prices:

Detect regime shifts between high/low volatility periods

Identify dates when mean price level changes significantly

Separate calm periods from crisis periods

How They Help:

Objective Identification: Data-driven detection vs. subjective event selection

Quantification: Measure magnitude of change (mean shift, variance change)

Timing Precision: Identify exact dates of structural breaks

Causal Inference Support: Provide evidence for event impact hypotheses

2.3 Expected Outputs and Limitations:
Expected Outputs:

Change Point Dates: Posterior distribution of τ (change point location)

Parameter Estimates: Distributions for μ₁ (mean before) and μ₂ (mean after)

Impact Quantification: Probability distribution of price change magnitude

Uncertainty Measures: Credible intervals for all estimates

Limitations:

Assumes at most one change point in the analyzed period

Sensitive to prior distributions

Requires careful convergence diagnostics

May miss gradual transitions